In [1]:
%load_ext rpy2.ipython

In [2]:
%%R
trainData = data.matrix(read.csv( file="train.csv", header=TRUE ))

In [3]:
%%R
print(head(trainData))

     bidder_id payment_account address outcome
[1,]      1159             824    1363       0
[2,]       774            1536    1480       0
[3,]       219             620     955       0
[4,]       623             315     507       0
[5,]       614             770     262       0
[6,]      1004            1645     595       0


In [4]:
# %%R
# bidsDataSet = data.matrix(read.csv( file="bids.csv", header=TRUE ))
# print(head(bidsDataSet))

In [5]:
# import csv
# import re

# #test two sample bidders

# bidsFile = open('bids.csv', 'rU')

# testBotBidder = '5354c02817e47f28e60e44a40ca5e48dll4sz'
# testHumBidder = '91a3c57b13234af24875c56fb7e2b2f4rb56a'

# foutBot = open('botBidder.csv', 'wb')
# foutHum = open('humanBidder.csv', 'wb')
# fin = open('bids.csv', 'rU')
# rdr = csv.reader(fin, delimiter=',')
# wtrBot = csv.writer(foutBot, delimiter=',')
# wtrHum = csv.writer(foutHum, delimiter=',')

# for row in rdr:
#     if row[1] == 'bidder_id':
#         wtrBot.writerow(row[2:])
#         wtrHum.writerow(row[2:])
#     if row[1] == testBotBidder:
#         wtrBot.writerow(row[2:])
#     if row[1] == testHumBidder:
#         wtrHum.writerow(row[2:])
#     if row[8] == "": #check if an empty cell exists
#         print row[1]
        
# foutBot.close()
# foutHum.close()
# fin.close()


In [6]:
import csv

#build a dictionary : bidder id -> class

print('starting ...')

fin = open('train.csv', 'rU')
rdr = csv.reader(fin, delimiter=',')

bidderIdToClass = {}

for row in rdr:
    bidderId = row[0]
    if row[3] == '1.0': #it is a bot
        bidderIdToClass[bidderId] = 1
    elif row[3] == '0.0':
        bidderIdToClass[bidderId] = 0

print(len(bidderIdToClass))

#1910 human bidders, 103 bot bidders (classified) ???

fin.close()

starting ...
2013


In [7]:
import csv

#separate bids file into bot_bids and hum_bids

print('starting ...')

fin = open('bids.csv', 'rU')
rdr = csv.reader(fin, delimiter=',')

botFout = open('bot_bids.csv', 'wb')
humFout = open('hum_bids.csv', 'wb')
trainFout = open('train_bids.csv', 'wb')

botWtr = csv.writer(botFout, delimiter=',')
humWtr = csv.writer(humFout, delimiter=',')
trainWtr = csv.writer(trainFout, delimiter=',')
dummyRowCount = 0

#write only the bidders which exist in training dataset
for row in rdr:
    bidderId = row[1]
    if row[0] == 'bid_id': #header
        botWtr.writerow(row)
        humWtr.writerow(row)
    elif bidderIdToClass.has_key(bidderId) and bidderIdToClass[bidderId] == 1:
        botWtr.writerow(row)
    elif bidderIdToClass.has_key(bidderId) and bidderIdToClass[bidderId] == 0:
        humWtr.writerow(row)
    else:
        dummyRowCount = dummyRowCount + 1
        
    if bidderIdToClass.has_key(bidderId): #write both hum and bot in training dataset
        trainWtr.writerow(row)
        
fin.close()
botFout.close()
humFout.close()
trainFout.close()

print(dummyRowCount)
print('done')

starting ...
4585110
done


In [7]:
import csv

#process bot bidders info

fin = open('bot_bids.csv', 'rU')
rdr = csv.reader(fin, delimiter=',')

# things to consider for each bidder
# -arrival distribution
# -min interval (with sliding window of a certain size)
# -total number of bids
# -number of devices used
# -number of auctions used
# -number of different merchandises (categories)
# -type of merchandises
# -number of countries accessed 
# -what time during the day
# -number of urls accessed from


#bidderIdToNumBids = {}
bidderIdToInfo = {} #bidderId -> info object

slidingWindowSize = 10

for row in rdr:
    bidderId = row[1]
    currArrivalTime = row[5]
    currMerchandise = row[3]
    currAuction = row[2]
    currURL = row[8]
    currDevice = row[4]
    currCountry = row[6]
    
    if bidderId == 'bidder_id':
        continue
    elif bidderIdToInfo.has_key(bidderId):
        #number of bids
        bidderIdToInfo[bidderId]['numBids'] = bidderIdToInfo[bidderId]['numBids'] + 1
        
        #interval
        arrivalTimesList = bidderIdToInfo[bidderId]['arrivalTimes']
        arrivalTimesList.append(currArrivalTime) #append the new arrival time
        length = len(arrivalTimesList)
        bidderIdToInfo[bidderId]['avgInterval'] = (int(currArrivalTime)-int(arrivalTimesList[0])) / (length-1)
        if length >= slidingWindowSize: #window size at least = 3
            currMin = (int(currArrivalTime) - int(arrivalTimesList[length-slidingWindowSize])) / (slidingWindowSize-1)
            if bidderIdToInfo[bidderId]['minInterval'] == '' or currMin < bidderIdToInfo[bidderId]['minInterval']:
                bidderIdToInfo[bidderId]['minInterval'] = currMin
                
        #merchandises
        merchandises = bidderIdToInfo[bidderId]['merchandises']
        if merchandises.has_key(currMerchandise):
            merchandises[currMerchandise] = merchandises[currMerchandise] + 1
        else:
            merchandises[currMerchandise] = 1
            
        #auctions
        auctions = bidderIdToInfo[bidderId]['auctions']
        if not auctions.has_key(currAuction):
            auctions[currAuction] = 1
        
        #urls
        urls = bidderIdToInfo[bidderId]['urls']
        if not urls.has_key(currURL):
            urls[currURL] = 1
            
        #devices
        devices = bidderIdToInfo[bidderId]['devices']
        if not devices.has_key(currDevice):
            devices[currDevice] = 1
        
        #countries
        countries = bidderIdToInfo[bidderId]['countries']
        if not countries.has_key(currCountry):
            countries[currCountry] = 1
        
    else:
        newInfo = {}
        newInfo['numBids'] = 1
        arrivalTimesList = []
        arrivalTimesList.append(currArrivalTime) #append the arrival time of the current bid
        newInfo['arrivalTimes'] = arrivalTimesList
        newInfo['avgInterval'] = ''
        newInfo['minInterval'] = ''
        
        newMerchandises = {}
        newMerchandises[currMerchandise] = 1
        newInfo['merchandises'] = newMerchandises
        
        newAuctions = {}
        newAuctions[currAuction] = 1
        newInfo['auctions'] = newAuctions
        
        newURLs = {}
        newURLs[currURL] = 1
        newInfo['urls'] = newURLs
        
        newDevices = {}
        newDevices[currDevice] = 1
        newInfo['devices'] = newDevices
        
        newCountries = {}
        newCountries[currCountry] = 1
        newInfo['countries'] = newCountries
        
        bidderIdToInfo[bidderId] = newInfo
        
fin.close()

#write the results
fout = open('bot_bidders_info.csv', 'wb')
wtr = csv.writer(fout, delimiter=',')

wtr.writerow(['bidder_id', 'number of bids', 'avg arrival interval', 'min interval', 'number of auctions', 'number of urls', 'merchandise', 'number of devices', 'number of countries'])
for key, val in bidderIdToInfo.items(): 
    merchList = val['merchandises'].keys()[0] #there exists only one merchandise per bidder
    currRowVec = [key, str(val['numBids']), val['avgInterval'], val['minInterval'], len(val['auctions']), len(val['urls']), merchList, len(val['devices']), len(val['countries'])]
    wtr.writerow(currRowVec)

# for key, val in bidderIdToNumBids.items():
#     currRowVec = [key, str(val)]
#     wtr.writerow(currRowVec)
    
fout.close()

print('Bot bidders: ' + str(len(bidderIdToInfo)))

Bot bidders: 103


In [8]:
import csv

#process human bidders info

fin = open('hum_bids.csv', 'rU')
rdr = csv.reader(fin, delimiter=',')

# things to consider for each bidder
# -arrival distribution
# -max rate
# -total number of bids
# -number of devices used
# -number of auctions used
# -number of different merchandises (categories)
# -number of countries accessed 


#bidderIdToNumBids = {}
bidderIdToInfo = {} #bidderId -> info object

for row in rdr:
    bidderId = row[1]
    currArrivalTime = row[5]
    currMerchandise = row[3]
    currAuction = row[2]
    currURL = row[8]
    currDevice = row[4]
    currCountry = row[6]
    
    if bidderId == 'bidder_id':
        continue
    elif bidderIdToInfo.has_key(bidderId):
        #number of bids
        bidderIdToInfo[bidderId]['numBids'] = bidderIdToInfo[bidderId]['numBids'] + 1
        
        #interval
        arrivalTimesList = bidderIdToInfo[bidderId]['arrivalTimes']
        arrivalTimesList.append(currArrivalTime) #append the new arrival time
        length = len(arrivalTimesList)
        bidderIdToInfo[bidderId]['avgInterval'] = (int(currArrivalTime)-int(arrivalTimesList[0])) / (length-1)
        if length >= slidingWindowSize: #window size at least = 3
            currMin = (int(currArrivalTime) - int(arrivalTimesList[length-slidingWindowSize])) / (slidingWindowSize-1)
            if bidderIdToInfo[bidderId]['minInterval'] == '' or currMin < bidderIdToInfo[bidderId]['minInterval']:
                bidderIdToInfo[bidderId]['minInterval'] = currMin
                
        #merchandises
        merchandises = bidderIdToInfo[bidderId]['merchandises']
        if merchandises.has_key(currMerchandise):
            merchandises[currMerchandise] = merchandises[currMerchandise] + 1
        else:
            merchandises[currMerchandise] = 1
            
        #auctions
        auctions = bidderIdToInfo[bidderId]['auctions']
        if not auctions.has_key(currAuction):
            auctions[currAuction] = 1
        
        #urls
        urls = bidderIdToInfo[bidderId]['urls']
        if not urls.has_key(currURL):
            urls[currURL] = 1
            
        #devices
        devices = bidderIdToInfo[bidderId]['devices']
        if not devices.has_key(currDevice):
            devices[currDevice] = 1
        
        #countries
        countries = bidderIdToInfo[bidderId]['countries']
        if not countries.has_key(currCountry):
            countries[currCountry] = 1
        
    else:
        newInfo = {}
        newInfo['numBids'] = 1
        arrivalTimesList = []
        arrivalTimesList.append(currArrivalTime) #append the arrival time of the current bid
        newInfo['arrivalTimes'] = arrivalTimesList
        newInfo['avgInterval'] = ''
        newInfo['minInterval'] = ''
        
        newMerchandises = {}
        newMerchandises[currMerchandise] = 1
        newInfo['merchandises'] = newMerchandises
        
        newAuctions = {}
        newAuctions[currAuction] = 1
        newInfo['auctions'] = newAuctions
        
        newURLs = {}
        newURLs[currURL] = 1
        newInfo['urls'] = newURLs
        
        newDevices = {}
        newDevices[currDevice] = 1
        newInfo['devices'] = newDevices
        
        newCountries = {}
        newCountries[currCountry] = 1
        newInfo['countries'] = newCountries
        
        bidderIdToInfo[bidderId] = newInfo
        
fin.close()

#write the results
fout = open('hum_bidders_info.csv', 'wb')
wtr = csv.writer(fout, delimiter=',')

wtr.writerow(['bidder_id', 'number of bids', 'avg arrival interval', 'min interval', 'number of auctions', 'number of urls', 'merchandise', 'number of devices', 'number of countries'])
for key, val in bidderIdToInfo.items(): 
    merchList = val['merchandises'].keys()[0] #there exists only one merchandise per bidder
    currRowVec = [key, str(val['numBids']), val['avgInterval'], val['minInterval'], len(val['auctions']), len(val['urls']), merchList, len(val['devices']), len(val['countries'])]
    wtr.writerow(currRowVec)

# for key, val in bidderIdToNumBids.items():
#     currRowVec = [key, str(val)]
#     wtr.writerow(currRowVec)
    
fout.close()

print('Hum bidders: ' + str(len(bidderIdToInfo)))

Hum bidders: 1881


In [9]:
import csv

#bidderIdToClass from train.csv
#2013 - 1881 - 103 = 29 leftovers
#verify the number of leftovers (that have no bid in bids.csv)

bidderIdToClassTemp = bidderIdToClass.copy()

fin = open('bids.csv', 'rU')
rdr = csv.reader(fin, delimiter=',')

for row in rdr:
    bidderId = row[1]
    if bidderIdToClassTemp.has_key(bidderId):
        del bidderIdToClassTemp[bidderId]
        
fin.close()
print(len(bidderIdToClassTemp))
print(len(bidderIdToClass))

29
2013


In [23]:
#reaction time for each bidder

import csv

print('starting ...')

fin = open('bids.csv', 'rU')
rdr = csv.reader(fin, delimiter=',')

bidderIdToReactionTimeList = {}
auctionToLastBid = {}
firstRow = True

for row in rdr:
    if firstRow:
        firstRow = False
        continue
    
    bidderId = row[1]
    auction = row[2]
    time = int(row[5])
    
    if not auctionToLastBid.has_key(auction):
        auctionToLastBid[auction] = time
        continue
        
    reactionTime = time - auctionToLastBid[auction]
    auctionToLastBid[auction] = time
        
    if not bidderIdToReactionTimeList.has_key(bidderId):
        newList = [reactionTime]
        bidderIdToReactionTimeList[bidderId] = newList
    else:
        reactTimeList = bidderIdToReactionTimeList[bidderId]
        reactTimeList.append(reactionTime)
    
        
fin.close()

print(len(bidderIdToReactionTimeList))
print('done')

starting ...
6610
done


In [27]:
#attach reaction time feature with bidderIdToReactionTimeList

import numpy
def median(lst):
    return numpy.median(numpy.array(lst))

#modify human info

fin = open('hum_bidders_info.csv', 'rU')
rdr = csv.reader(fin, delimiter=',')
fout = open('hum_bidders_info_reaction.csv', 'wb')
wtr = csv.writer(fout, delimiter=',')

firstRow = True
for row in rdr:
    if firstRow:
        firstRow = False
        row.append('min reaction time')
        row.append('median reaction time')
        wtr.writerow(row)
        continue
        
    bidderId = row[0]
    
    if not bidderIdToReactionTimeList.has_key(bidderId): #if the bidder made only one bid as the first in the auction
        print bidderId
        wtr.writerow(row)
        continue
    
    reactionTimeList = bidderIdToReactionTimeList[bidderId]
    row.append(min(reactionTimeList))
    row.append(median(reactionTimeList))
    wtr.writerow(row)

fin.close()
fout.close()

#modify bot info

fin = open('bot_bidders_info.csv', 'rU')
rdr = csv.reader(fin, delimiter=',')
fout = open('bot_bidders_info_reaction.csv', 'wb')
wtr = csv.writer(fout, delimiter=',')

firstRow = True
for row in rdr:
    if firstRow:
        firstRow = False
        row.append('min reaction time')
        row.append('median reaction time')
        wtr.writerow(row)
        continue
        
    bidderId = row[0]
    
    if not bidderIdToReactionTimeList.has_key(bidderId):
        print bidderId
        wtr.writerow(row)
        continue
    
    reactionTimeList = bidderIdToReactionTimeList[bidderId]
    row.append(min(reactionTimeList))
    row.append(median(reactionTimeList))
    wtr.writerow(row)

fin.close()
fout.close()

print("done")

ca391c857d7aa75a72e2d7c3133e0d6ay9pil
done


In [32]:
#merge

fin = open('bot_bidders_info_reaction.csv', 'rU')
rdr = csv.reader(fin, delimiter=',')

fout = open('train_bidders_info_final.csv', 'wb')
wtr = csv.writer(fout, delimiter=',')

firstRow = True

#bot first
for row in rdr:
    if firstRow:
        firstRow = False
        row.append('class')
        wtr.writerow(row)
        continue
    row.append(1.0)
    wtr.writerow(row)

firstRow = True
    
#now human
fin.close()
fin = open('hum_bidders_info_reaction.csv', 'rU')
rdr = csv.reader(fin, delimiter=',')
for row in rdr:
    if firstRow:
        firstRow = False
        continue
    row.append(0.0)
    wtr.writerow(row)
    
fin.close()
fout.close()

print("merging done")

merging done
